<a href="https://colab.research.google.com/github/antoronson/Vision-Based-Tracking/blob/dev/Vision_Based_Tracking/Object_Tracking_From_Video_Stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from IPython.display import HTML

from base64 import b64encode

def play_mp4(file_path):
  mp4 = open(file_path, 'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

  # Add "return" here!
  return HTML(f"""
  <video width="640" height="480" controls>
      <source src="{data_url}" type="video/mp4">
  </video>
  """)

In [ ]:
model = YOLO('yolov8n.pt')

dir = "/content/drive/MyDrive/datasets/yolo_video"

input_file = "input_file.mp4"

output_file = "output_file.mp4"

input_path = os.path.join(dir, input_file)

output_path = os.path.join(dir, output_file)

play_mp4(input_path)